In [ ]:
# Pull traffic data from Chicago City Data API
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import numpy as np

# Import downloaded csv file
crashes_csv = "Traffic_Crashes.csv"
traffic_crashes = pd.read_csv(crashes_csv)
crashes_df = pd.DataFrame(traffic_crashes)
crashes_drop_df = crashes_df.dropna(subset=['LATITUDE', 'LONGITUDE'], how='all')
crashes_drop_df.head()

In [ ]:
crashes_drop_df.columns

In [ ]:
# Reduce to only columns we care about
crashes_drop_df = crashes_drop_df[['CRASH_DATE', 'WEATHER_CONDITION', 'LIGHTING_CONDITION',
                                   'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'FIRST_CRASH_TYPE','PRIM_CONTRIBUTORY_CAUSE',
                                   'INJURIES_TOTAL','INJURIES_FATAL', 'CRASH_HOUR','CRASH_DAY_OF_WEEK',
                                   'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION']]


# Reformat the date column to be easily comparable dates
crashes_drop_df['CRASH_DATE'] = pd.to_datetime(crashes_drop_df['CRASH_DATE'])
crashes_drop_df.sort_values(by='CRASH_DATE', ascending=False)

# Cut Lat and Lon to 4 decimal places
# crashes_drop_df['LATITUDE'] = crashes_drop_df['LATITUDE'].round(3)
# crashes_drop_df['LONGITUDE'] = crashes_drop_df['LONGITUDE'].round(3)


# Filter the data frame for only dates 2018 onward

crashes_2017_18 = crashes_drop_df.loc[(crashes_drop_df['CRASH_DATE'] >= "2017-01-01") & 
                                        (crashes_drop_df['CRASH_DATE'] <= "2018-12-31"), :]

crashes_2017_18.head()


In [ ]:
# Make a data frame with the Chicago population from 2015 to 2019

pop_years = [2014, 2015, 2016, 2017, 2018]
chi_pops = [2728524, 2726215, 2718946, 2713067, 2705994]
chi_yearly_pops = {"Year": pop_years, "Chicago Population": chi_pops}

chicago_pop_df = pd.DataFrame(chi_yearly_pops)
chicago_pop_df

In [ ]:
# Filter original dataframe for only crashes with a fatality
fatal_crashes = crashes_2017_18.loc[(crashes_2017_18['INJURIES_FATAL'] > 0), :]

fatal_groupby = fatal_crashes.groupby(['FIRST_CRASH_TYPE', 'LIGHTING_CONDITION'])
fatal_groupby.count().sort_values(by='Crash Date', ascending=False).head(10)

In [ ]:
# Import Rideshare CSV
rideshare_csv = "rs_filter_csv.csv"
rideshare_data = pd.read_csv(rideshare_csv)
rideshare_df = pd.DataFrame(rideshare_data)

rideshare_df = rideshare_df[['Month', 'NUMBER_OF_TRIPS_2015', 'NUMBER_OF_TRIPS_2016', 'NUMBER_OF_TRIPS_2017',
                            'NUMBER_OF_TRIPS_2018']]

rideshare_df

In [ ]:
# Round crashes_2017_18 to three decimals, and merge with speed cam data

crashes_2017_18['LATITUDE'] = crashes_2017_18['LATITUDE'].round(3)
crashes_2017_18['LONGITUDE'] = crashes_2017_18['LONGITUDE'].round(3)

accidents_speedcams = crashes_2017_18.merge(speed_df, how="inner", on=('LATITUDE', 'LONGITUDE'))
accidents_speedcams

In [ ]:
accidents_speedcams.columns

In [ ]:
# Select desired columns and filter on 2017 and 2018
accidents_speedcams = accidents_speedcams[['Crash Date', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'FIRST_CRASH_TYPE',
                                           'INJURIES_TOTAL', 'PRIM_CONTRIBUTORY_CAUSE', 'INJURIES_FATAL',
                                           'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE',
                                           'LONGITUDE', 'CAMERA ID', 'VIOLATIONS', 'VIOLATION DATE']]

accidents_speedcams = accidents_speedcams.loc[(accidents_speedcams['Crash Date'] >= pd.to_datetime("2017-01-01").date()) & 
                                        (accidents_speedcams['Crash Date'] <= pd.to_datetime("2018-12-31").date()), :]

accidents_speedcams

In [ ]:
# Group crashes by their proximity to speed cameras
accidents_cameras_group = accidents_speedcams.groupby(['CAMERA ID'])
accidents_cameras_count = acc_speed_group.nunique().sort_values(by='Crash Date', ascending=False)

accidents_cameras_count = accidents_cameras_count[['Crash Date']] 
accidents_cameras_count.head(10)

In [ ]:
#speed camera
import pandas as pd
from sodapy import Socrata
import datetime
import gmaps
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.lines as mlines
import gmaps.datasets
import scipy.stats as stats
import folium
from folium import plugins
import seaborn as sns; sns.set()



In [23]:
#read dataframe
speed_df = pd.read_csv('Speed_Camera_Violations.csv')
speed_df.head()

,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,10318 S INDIANAPOLIS,CHI120,06/10/2019,83,1.203645e+06,1.837056e+06,41.707577,-87.529848,"(41.70757690291348, -87.52984826112849)",47.0,21202.0,49.0,705.0,47.0
1,1110 S PULASKI RD,CHI163,06/10/2019,23,1.149841e+06,1.894931e+06,41.867603,-87.725383,"(41.86760272243294, -87.7253827433152)",36.0,21572.0,30.0,98.0,14.0
2,1111 N HUMBOLDT,CHI010,06/10/2019,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11144 S VINCENNES,CHI023,06/10/2019,34,1.166994e+06,1.830711e+06,41.691025,-87.664248,"(41.69102545584918, -87.6642476900556)",33.0,22212.0,74.0,378.0,42.0
4,11153 S VINCENNES,CHI022,06/10/2019,10,1.167029e+06,1.830594e+06,41.690702,-87.664122,"(41.690701951255015, -87.66412238501842)",45.0,22212.0,74.0,378.0,22.0


In [3]:
#change to datetime, then drop NaN
speed_df['VIOLATION DATE'] = pd.to_datetime(speed_df['VIOLATION DATE'])
speed_df_drop = speed_df.dropna()

In [4]:
#uber dataframe
uber_df = pd.read_csv('rs_filter_csv.csv')
uber_df.head()

,MONTH_REPORTED,NUMBER_OF_TRIPS_2015,NUMBER_OF_TRIPS_2016,NUMBER_OF_TRIPS_2017,NUMBER_OF_TRIPS_2018
0,1,518539,1733583,3582540,4842911
1,2,518539,1732573,3455601,4718735
2,3,712521,1887034,4017804,5284216
3,4,764487,2014292,3839776,3500907
4,5,868650,2184055,4039394,3580748


In [5]:
# filtered by date
speed_18 = speed_df_drop[(speed_df_drop['VIOLATION DATE']>=datetime.date(2018,1,1)) & (speed_df_drop['VIOLATION DATE']<=datetime.date(2018,12,31))]
speed_17 = speed_df_drop[(speed_df_drop['VIOLATION DATE']>=datetime.date(2017,1,1)) & (speed_df_drop['VIOLATION DATE']<=datetime.date(2017,12,31))]
speed_16 = speed_df_drop[(speed_df_drop['VIOLATION DATE']>=datetime.date(2016,1,1)) & (speed_df_drop['VIOLATION DATE']<=datetime.date(2016,12,31))]
speed_15 = speed_df_drop[(speed_df_drop['VIOLATION DATE']>=datetime.date(2015,1,1)) & (speed_df_drop['VIOLATION DATE']<=datetime.date(2015,12,31))]
speed_14 = speed_df_drop[(speed_df_drop['VIOLATION DATE']>=datetime.date(2014,1,1)) & (speed_df_drop['VIOLATION DATE']<=datetime.date(2014,12,31))]

C:\Users\consi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  
C:\Users\consi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\consi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the

In [6]:
#groupby
speed_camera_18 = speed_18.groupby(['CAMERA ID','LATITUDE','LONGITUDE'])
speed_camera_17 = speed_17.groupby(['CAMERA ID','LATITUDE','LONGITUDE'])
speed_camera_16 = speed_16.groupby(['CAMERA ID','LATITUDE','LONGITUDE'])
speed_camera_15 = speed_15.groupby(['CAMERA ID','LATITUDE','LONGITUDE'])
speed_camera_14 = speed_14.groupby(['CAMERA ID','LATITUDE','LONGITUDE'])

In [11]:
#sort18
sorted_speed_18 = speed_camera_18["VIOLATIONS"].sum().sort_values(ascending=False).reset_index()
top_20_18 = sorted_speed_18.head(20)
#sort17
sorted_speed_17 = speed_camera_17["VIOLATIONS"].sum().sort_values(ascending=False).reset_index()
top_20_17 = sorted_speed_17.head(20)
#sort16
sorted_speed_16 = speed_camera_16["VIOLATIONS"].sum().sort_values(ascending=False).reset_index()
top_20_16 = sorted_speed_16.head(20)
#sort15
sorted_speed_15 = speed_camera_15["VIOLATIONS"].sum().sort_values(ascending=False).reset_index()
top_20_15 = sorted_speed_15.head(20)
#sort14
sorted_speed_14 = speed_camera_14["VIOLATIONS"].sum().sort_values(ascending=False).reset_index()
top_20_14 = sorted_speed_14.head(20)


In [12]:
#total violations per year
violations_18 = sorted_speed_18['VIOLATIONS'].sum()
violations_17 = sorted_speed_17['VIOLATIONS'].sum()
violations_16 = sorted_speed_16['VIOLATIONS'].sum()
violations_15 = sorted_speed_15['VIOLATIONS'].sum()
violations_14 = sorted_speed_14['VIOLATIONS'].sum()
#total violations dataframe
total_violations = pd.DataFrame({
    '2014':[violations_14],
    '2015':[violations_15],
    '2016':[violations_16],
    '2017':[violations_17],
    '2018':[violations_18]
})
vio = total_violations.T
vio

,0
2014,630856
2015,1157460
2016,1077135
2017,976290
2018,927820


In [15]:
#uber and total violation dataframe

uber_total_15 = uber_df['NUMBER_OF_TRIPS_2015'].sum() 
uber_total_16 = uber_df['NUMBER_OF_TRIPS_2016'].sum() 
uber_total_17 = uber_df['NUMBER_OF_TRIPS_2017'].sum() 
uber_total_18 = uber_df['NUMBER_OF_TRIPS_2018'].sum() 
uber_total = pd.DataFrame({
    '2015':[uber_total_15],
    '2016':[uber_total_16],
    '2017':[uber_total_17],
    '2018': [uber_total_18]
})

violation_total = pd.DataFrame({
    
    '2015':[violations_15],
    '2016':[violations_16],
    '2017':[violations_17],
    '2018':[violations_18]
})
print(violation_total.mean())
print(uber_total.mean())

2015    1157460.0
2016    1077135.0
2017     976290.0
2018     927820.0
dtype: float64
2015    11052156.0
2016    29898808.0
2017    48857182.0
2018    57779874.0
dtype: float64


In [16]:
#line graph data
speed_camera_18_date = speed_18.groupby(['VIOLATION DATE'])

date_18 = speed_camera_18_date['VIOLATIONS'].sum().reset_index()

speed_camera_17_date = speed_17.groupby(['VIOLATION DATE'])

date_17 = speed_camera_17_date['VIOLATIONS'].sum().reset_index()

speed_camera_16_date = speed_16.groupby(['VIOLATION DATE'])

date_16 = speed_camera_16_date['VIOLATIONS'].sum().reset_index()

speed_camera_15_date = speed_15.groupby(['VIOLATION DATE'])

date_15 = speed_camera_15_date['VIOLATIONS'].sum().reset_index()

speed_camera_14_date = speed_14.groupby(['VIOLATION DATE'])

date_14 = speed_camera_14_date['VIOLATIONS'].sum().reset_index()


In [17]:
#map info total
total_group = speed_df.groupby(['CAMERA ID','LATITUDE','LONGITUDE'])
violations_total = total_group['VIOLATIONS'].sum().reset_index()

locations_total= violations_total[["LATITUDE","LONGITUDE"]]
rating_total =violations_total["VIOLATIONS"].astype(float)

chicago_center = (41.8781,-87.6298)

In [18]:
# Top 20 cameras from 2014
top_20 = top_20_14['CAMERA ID'].to_frame()

In [19]:
#top 20 cameras from 2014 merged into the 2015 through 2018

merger_15 = sorted_speed_15.merge(top_20, how = 'inner').sort_values('VIOLATIONS')
merger_16 = sorted_speed_16.merge(top_20, how = 'inner').sort_values('VIOLATIONS')
merger_17 = sorted_speed_17.merge(top_20, how = 'inner').sort_values('VIOLATIONS')
merger_18 = sorted_speed_18.merge(top_20, how = 'inner').sort_values('VIOLATIONS')
merg_15 = merger_15.rename(columns = ({'VIOLATIONS':'Violations_2015'}))
merg_16 = merger_16.rename(columns = ({'VIOLATIONS':'Violations_2016'}))
merg_17 = merger_17.rename(columns = ({'VIOLATIONS':'Violations_2017'}))
merg_18 = merger_18.rename(columns = ({'VIOLATIONS':'Violations_2018'}))

#form into a dataframe

top = top_20_14.rename(columns= {'VIOLATIONS':'Violations_2014'}).sort_values('Violations_2014')
violations_1 = top.merge(merg_15, how='inner')
violations_11 = violations_1.merge(merg_16, how = 'inner')
violations_111 = violations_11.merge(merg_17, how = 'inner')
violations_final = violations_111.merge(merg_18, how = 'inner')
violations_final['Total']=violations_final['Violations_2014']+violations_final['Violations_2015']+violations_final['Violations_2016']+violations_final['Violations_2017']+violations_final['Violations_2018']
violations_finals = violations_final.sort_values('Total')

In [20]:
years = 2015,2016,2017,2018
chi_pops = [2726215, 2718946, 2713067, 2705994]

chicago_pop_df = pd.DataFrame({
    'Year':years,
    'Pop':chi_pops
})


print(violation_total.mean())
chicago_pop_df['Pop']

2015    1157460.0
2016    1077135.0
2017     976290.0
2018     927820.0
dtype: float64


0    2726215
1    2718946
2    2713067
3    2705994
Name: Pop, dtype: int64

In [21]:
stats.pearsonr(chicago_pop_df['Pop'], violation_total.mean())

(0.9880440201257168, 0.011955979874283209)

In [22]:
stats.pearsonr(uber_total.mean(),violation_total.mean())

(-0.9984176655162007, 0.0015823344837992976)